In [0]:
file_loc = "/FileStore/tables/tips.csv"

In [0]:
df = spark.read.csv(file_loc,header = True,inferSchema = True)

In [0]:
df.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [0]:
df.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)



In [0]:
df.columns

Out[5]: ['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size']

In [0]:
# encoding columns
from pyspark.ml.feature import StringIndexer

In [0]:
indexer = StringIndexer(inputCols=["sex","smoker","day","time"],outputCols=["sex_index","smoker_index","day_index","time_index"])
df1=indexer.fit(df).transform(df)
df1.show()

+----------+----+------+------+---+------+----+---------+------------+---------+----------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_index|smoker_index|day_index|time_index|
+----------+----+------+------+---+------+----+---------+------------+---------+----------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|      1.0|         0.0|      1.0|       0.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|      0.0|         0.0|      1.0|       0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|      0.0|         0.0|      1.0|       0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|      0.0|         0.0|      1.0|       0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|      1.0|         0.0|      1.0|       0.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|      0.0|         0.0|      1.0|       0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|      0.0|         0.0|      1.0|       0.0|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|      0.0|         0.0|      1.0|

In [0]:
from pyspark.ml.feature import VectorAssembler 
featureassembler = VectorAssembler(inputCols= ['tip',"sex_index","smoker_index","day_index","time_index"], outputCol="Independent Features")

output = featureassembler.transform(df1)

In [0]:
finalized_data = output.select("Independent Features","total_bill")


In [0]:
finalized_data.show()

+--------------------+----------+
|Independent Features|total_bill|
+--------------------+----------+
|[1.01,1.0,0.0,1.0...|     16.99|
|(5,[0,3],[1.66,1.0])|     10.34|
| (5,[0,3],[3.5,1.0])|     21.01|
|(5,[0,3],[3.31,1.0])|     23.68|
|[3.61,1.0,0.0,1.0...|     24.59|
|(5,[0,3],[4.71,1.0])|     25.29|
| (5,[0,3],[2.0,1.0])|      8.77|
|(5,[0,3],[3.12,1.0])|     26.88|
|(5,[0,3],[1.96,1.0])|     15.04|
|(5,[0,3],[3.23,1.0])|     14.78|
|(5,[0,3],[1.71,1.0])|     10.27|
|[5.0,1.0,0.0,1.0,...|     35.26|
|(5,[0,3],[1.57,1.0])|     15.42|
| (5,[0,3],[3.0,1.0])|     18.43|
|[3.02,1.0,0.0,1.0...|     14.83|
|(5,[0,3],[3.92,1.0])|     21.58|
|[1.67,1.0,0.0,1.0...|     10.33|
|(5,[0,3],[3.71,1.0])|     16.29|
|[3.5,1.0,0.0,1.0,...|     16.97|
|      (5,[0],[3.35])|     20.65|
+--------------------+----------+
only showing top 20 rows



In [0]:
from pyspark.ml.regression import LinearRegression
# train test split
train_data,test_data = finalized_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol='Independent Features', labelCol='total_bill')
regressor = regressor.fit(train_data)

In [0]:
regressor.coefficients

Out[16]: DenseVector([4.2995, -1.6026, 0.3312, -0.6607, -0.7382])

In [0]:
regressor.intercept

Out[17]: 8.402621375752965

In [0]:
#predictions
pred_results = regressor.evaluate(test_data)

In [0]:
pred_results.predictions.show()

+--------------------+----------+------------------+
|Independent Features|total_bill|        prediction|
+--------------------+----------+------------------+
|      (5,[0],[1.25])|     10.51|13.777033106685895|
|      (5,[0],[1.45])|      9.55|14.636938983635163|
|      (5,[0],[1.47])|     10.77| 14.72292957133009|
|      (5,[0],[1.75])|     17.82|15.926797799059067|
|      (5,[0],[2.24])|     16.04|18.033567197584773|
|      (5,[0],[2.34])|     17.81|18.463520136059408|
|      (5,[0],[3.15])|     20.08| 21.94613893770395|
|      (5,[0],[3.18])|     19.82| 22.07512481924634|
|      (5,[0],[3.27])|     17.78|22.462082463873507|
|       (5,[0],[3.6])|     24.06|23.880927160839803|
| (5,[0,1],[1.0,1.0])|      7.25| 11.09956800844155|
|(5,[0,1],[2.23,1.0])|     15.77| 16.38798915167955|
|(5,[0,2],[1.17,1.0])|     32.83|13.764265730279373|
| (5,[0,2],[2.0,1.0])|     22.67| 17.33287511961884|
| (5,[0,2],[3.0,1.0])|     15.69|21.632404504365184|
| (5,[0,2],[3.0,1.0])|     38.73|21.6324045043

In [0]:
pred_results.r2,pred_results.meanAbsoluteError,pred_results.meanSquaredError

Out[20]: (0.34745529902911165, 4.996022408905782, 48.130073772203566)